In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Import MINST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

min_cost = 0.000001

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [3]:
def auto_encoder(x, input_size, output_size):
    weight = tf.Variable(tf.random_normal([input_size, output_size]))
    bias = tf.Variable(tf.random_normal([output_size]))
    
    encoder = tf.nn.sigmoid(tf.add(tf.matmul(x, weight), bias))
    
    weight2 = tf.transpose(weight)
    bias2 = tf.Variable(tf.random_normal([input_size]))
    
    decoder = tf.nn.sigmoid(tf.add(tf.matmul(encoder, weight2), bias2))
    
    return {
        'encoder':encoder,
        'weight':weight,
        'bias':bias,
        'decoder':decoder
    }

In [4]:
def train_encoder(optimiser, cost, input_data, batch_size, num_of_epoch):
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        
        input_size = len(input_data)
        
        total_batches = int(input_size/batch_size)
        
        for epoch in range(num_of_epoch):
            count = 0
            
            lowest_c = 100
            for i in range(total_batches):
                end = count + batch_size
                if end > input_size:
                    end = input_size
                
                batch_xs = input_data[count:end]
                
                _, c = sess.run([optimiser, cost], feed_dict={inputX:batch_xs})
                
                if c < lowest_c :
                    lowest_c = c
                
                count = end
            
            if epoch % 1 == 0:
                print("Epoch:", '%04d' % (epoch+1),
                      "cost=", "{:.9f}".format(c))
            
            if lowest_c < min_cost:
                break
            
        print("finished")       

In [5]:
def update_input_data(input_data, next_encoder_layer, size, batch_size):
    input_size = len(input_data)
    encoder = next_encoder_layer['encoder']

    output_data = np.empty([input_size, size])
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())


        total_batches = int(input_size / batch_size)
        count = 0
        for i in range(total_batches):
            end = count + batch_size
            if end > input_size:
                end = input_size

            batch_xs = input_data[count:end]
            output = sess.run(encoder, feed_dict={inputX: batch_xs})

            output_data[count:end] = output

    return output_data

In [6]:
def stack_auto_encoders(inputX, input_data, steps, batch_size, num_of_epoch):
    input_len = len(input_data[1])
    
    layers = []

    for step in range(len(steps)):
        next_encoder_layer = auto_encoder(inputX, input_len, steps[step])

        y_true = next_encoder_layer['decoder']
        y_pred = inputX

        cost = tf.reduce_mean(tf.pow(y_true - y_pred, 2))
        optimizer = tf.train.RMSPropOptimizer(0.01).minimize(cost)

        train_encoder(optimizer, cost, input_data, batch_size, num_of_epoch)

        input_data = update_input_data(input_data, next_encoder_layer,steps[step], batch_size)
        
        layers.append(next_encoder_layer)

        input_len = len(input_data[1])
    
    return layers
    

In [8]:
input_len = len(mnist.train.images[1])

inputX = tf.placeholder("float", shape=None)

layers = stack_auto_encoders(inputX, mnist.train.images, [256,128], 100, 20)

Epoch: 0001 cost= 0.032992233
Epoch: 0002 cost= 0.026079044
Epoch: 0003 cost= 0.023249928
Epoch: 0004 cost= 0.021445094
Epoch: 0005 cost= 0.018453483
Epoch: 0006 cost= 0.016957588
Epoch: 0007 cost= 0.015669139
Epoch: 0008 cost= 0.014795605
Epoch: 0009 cost= 0.014111629
Epoch: 0010 cost= 0.012687496
Epoch: 0011 cost= 0.009863646
Epoch: 0012 cost= 0.008498004
Epoch: 0013 cost= 0.007425163
Epoch: 0014 cost= 0.007128083
Epoch: 0015 cost= 0.006925079
Epoch: 0016 cost= 0.006779516
Epoch: 0017 cost= 0.006660591
Epoch: 0018 cost= 0.006076830
Epoch: 0019 cost= 0.005974868
Epoch: 0020 cost= 0.005874944
finished
Epoch: 0001 cost= 0.027345749
Epoch: 0002 cost= 0.019531835
Epoch: 0003 cost= 0.019531598
Epoch: 0004 cost= 0.015626185
Epoch: 0005 cost= 0.015625391
Epoch: 0006 cost= 0.015625335
Epoch: 0007 cost= 0.015625257
Epoch: 0008 cost= 0.015624418
Epoch: 0009 cost= 0.011719054
Epoch: 0010 cost= 0.011719024
Epoch: 0011 cost= 0.011718976
Epoch: 0012 cost= 0.011718933
Epoch: 0013 cost= 0.011718905
E